In [18]:
import pandas as pd
import sys
sys.path.append(r'C:\Users\Usuario\Desktop\Learning-Testing\PyStrava')
sys.path.append(r'C:\Users\Usuario\Desktop\Learning-Testing\PyStrava\Scripts')
from Functions import *
from StravaCredentials import hostname, database, username, pwd, port_id
import json
from sqlalchemy import create_engine

In [19]:
db_url = f'postgresql://{username}:{pwd}@{hostname}:{port_id}/{database}'
engine = create_engine(db_url)

In [2]:
conn = psycopg2.connect(
    host = hostname,
    dbname = database,
    user = username,
    password = pwd,
    port = port_id
)

cur = conn.cursor()

In [3]:
json_file = r'C:\Users\Usuario\Desktop\Learning-Testing\PyStrava\Outputs\all_workouts_json.json'
with open(json_file) as data:
    workout_json = json.load(data)

In [6]:
# Cleaning json and creating score columns
workout_df = CleanWorkoutJson(workout_json)
laps_df = CleanLapsJson(workout_json)
workout_df = CreateScoreColumns(workout_df)

In [20]:
# Breaking down workout_df to all different tables - laps table is = to laps_df
activity = workout_df[['activity_id',
                       'start_date',
                       'sport_type',
                       'effort_score_label']]

activity_name = workout_df[['activity_id',
                            'name']]

activity_coordinates = workout_df[['activity_id',
                                   'start_lat',
                                   'start_long',
                                   'end_lat',
                                   'end_long']]

activity_details = workout_df[['activity_id',
                               'workout_time_min',
                               'calories',
                               'average_heartrate',
                               'avg_lap_time',
                               'total_elevation_gain',
                               'max_heartrate',
                               'distance',
                               'average_speed_km/h',
                               'max_speed_km/h',
                               'average_temp',
                               'pace',
                               'lap_count']]

activity_scores = workout_df[['activity_id',
                              'distance_score',
                              'workout_time_score',
                              'calorie_score',
                              'total_elevation_gain_score',
                              'average_heartrate_score',
                              'max_heartrate_score',
                              'avg_lap_time_score',
                              'lap_count_score',
                              'avg_speed_score',
                              'max_speed_score',
                              'pace_score',
                              'effort_score',
                              'effort_score_rank']]

In [29]:
# Sending data to Postgresql
activity.to_sql('activity', engine, if_exists='replace', index=False)
activity_name.to_sql('activity_name', engine, if_exists='replace', index=False)
activity_coordinates.to_sql('activity_coordinates', engine, if_exists='replace', index=False)
activity_details.to_sql('activity_details', engine, if_exists='replace', index=False)
activity_scores.to_sql('activity_scores', engine, if_exists='replace', index=False)
laps_df.to_sql('laps', engine, if_exists='replace', index=False)

129

In [25]:
activity_name

,activity_id,name
0,10343078755,Morning Run
1,10322687345,Morning Run
2,10309277702,Evening Run
3,10298502876,Evening Run
4,10292826019,Evening Run
5,10257445914,Morning Run
6,10251037068,Morning Run
7,10233971826,Morning Run
8,10230744464,Afternoon Run
9,10220071235,Evening Run


20